In [1]:
import json

import urllib.request
import urllib.parse
import pandas as pd
import numpy as np
import time
from datetime import datetime
import re
import warnings
import time

pd.set_option('display.float_format',lambda x:'%.2f'%x)
# pd.set_option('display.max_columns',None)

In [2]:
fips = pd.read_excel('all-geocodes-v2016.xlsx',skiprows=[0,1,2,3], dtype = str)
fips = fips[fips['Summary Level']=='040']
fips['Area Name (including legal/statistical area description)'] = fips['Area Name (including legal/statistical area description)'].str.upper()
fips = fips[['State Code (FIPS)','Area Name (including legal/statistical area description)']]

##### State Income Tax Data

In [26]:

sheet_list = list(range(2000,2023))
state_income_tax = pd.DataFrame(columns = ['State','Low','High','year'])
for sheet in sheet_list:
    tax = pd.read_excel('state_income_tax_rates.xlsx', skiprows=list(range(0,5)),usecols=['State','Low','High'],nrows=51,sheet_name=str(sheet))
    tax['State'] = tax['State'].str.extract(r'([a-zA-Z+]*[ a-zA-Zw+]*)')
    tax['State'] = tax['State'].str.strip().str.upper()
    tax = tax.fillna(0)
    tax['year'] = sheet
    state_income_tax = state_income_tax.append(tax)
    
state_income_tax['Low'] = state_income_tax['Low'].astype(float)
state_income_tax['High'] = state_income_tax['High'].astype(float)
state_income_tax['year'] = state_income_tax['year'].astype(int)
#Add FIPS code to state
state_income_tax = state_income_tax.merge(fips, how='left',left_on = 'State',right_on = 'Area Name (including legal/statistical area description)' )
state_income_tax = state_income_tax.drop('Area Name (including legal/statistical area description)',axis=1)
state_income_tax = state_income_tax.rename({'State Code (FIPS)':'State_FIPS'},axis=1)
state_income_tax = state_income_tax[['State','State_FIPS', 'Low', 'High', 'year']]

# Uncomment to represent data in wide format using pivot table
# state_income_tax = state_income_tax.pivot_table(index=['State','State_FIPS'], columns=['year'])
# state_income_tax.columns = [str(a)+'_'+str(b) for a,b in state_income_tax.columns]


In [ ]:
state_income_tax.to_csv('state_income_tax_clean_year.csv',encoding='utf-8')

#### # State Corporate Income Tax Data

In [31]:
sheet_list = list(range(2002,2023))
sheet_list.remove(2009)
state_corp_income_tax = pd.DataFrame(columns = ['State','Rate','year'])
for sheet in sheet_list:
    tax = pd.read_excel('state_corporate_income_tax.xlsx', skiprows=list(range(0,5)),usecols=['State','Rate'],nrows=51,sheet_name=str(sheet))
    tax['Rate'] = tax['Rate'].astype(str)
    rates = tax['Rate'].str.extract(r'(?P<Range>(?P<Low>\d+.?\d*)(\s-\s*)(?P<High>\d+.?\d*))|(?P<Flat>\d+.?\d*)').fillna(0)
    rates['Low'] = rates[['Low','Flat']].astype(float).max(axis=1)
    rates['High'] = rates[['High','Flat']].astype(float).max(axis=1)
    rates = rates[['Low','High']]
    tax = tax.merge(rates,left_index=True, right_index=True)
    tax['State'] = tax['State'].str.extract(r'([a-zA-Z+]*[ a-zA-Zw+]*)')
    tax['State'] = tax['State'].str.strip().str.upper()
    tax = tax.dropna()
    tax['year'] = sheet
    state_corp_income_tax = state_corp_income_tax.append(tax)
    

state_corp_income_tax['year'] = state_corp_income_tax['year'].astype(int)
state_corp_income_tax = state_corp_income_tax.drop(['Rate'],axis=1)
#Add FIPS code to state
state_corp_income_tax = state_corp_income_tax.merge(fips, how='left',left_on = 'State',right_on = 'Area Name (including legal/statistical area description)' )
state_corp_income_tax = state_corp_income_tax.drop('Area Name (including legal/statistical area description)',axis=1)
state_corp_income_tax = state_corp_income_tax.rename({'State Code (FIPS)':'State_FIPS'},axis=1)
state_corp_income_tax = state_corp_income_tax[['State','State_FIPS', 'Low', 'High', 'year']]

# Uncomment to represent data in wide format using pivot table
# state_corp_income_tax = state_corp_income_tax.pivot_table(index=['State','State_FIPS'], columns=['year'])
# state_corp_income_tax.columns = [str(a)+'_'+str(b) for a,b in state_corp_income_tax.columns]

In [ ]:
state_corp_income_tax.to_csv('state_corp_income_tax_clean.csv',encoding='utf-8')

#### State and county debt ratio data

In [66]:
debt = pd.read_csv('household-debt-by-county.csv', dtype = str)
debt['State_FIPS'] = debt['area_fips'].apply(lambda x: x[0:2])
debt['County_FIPS'] = debt['area_fips'].apply(lambda x: x[2:5])
debt['low'] = debt['low'].astype(float)
debt['high'] = debt['high'].astype(float)
debt['high'][debt['high'].isna()]=debt['low']
debt = debt.groupby(['State_FIPS','County_FIPS','year'],as_index=False)['low','high'].mean()

# Uncomment to represent data in wide format using pivot table
# debt = debt.pivot_table(index=['State_FIPS','County_FIPS'], columns=['year']).reset_index()
# debt.columns = [str(a)+'_'+str(b) for a,b in debt.columns]

<ipython-input-66-5a2662e8ba31>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  debt['high'][debt['high'].isna()]=debt['low']
<ipython-input-66-5a2662e8ba31>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  debt = debt.groupby(['State_FIPS','County_FIPS','year'],as_index=False)['low','high'].mean()


In [68]:
debt.to_csv('debt_ratio_state_county_year_clean.csv',encoding='utf-8')